In [138]:
# imports
from z3 import *
from utils import *
import numpy as np

In [139]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 3, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance0 = {
    "m" : 2,
    "n" : 2,
    "l" : [15, 10],
    "s" : [3, 2],
    "distances" : [
        [0, 3, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instance2 = {
    "m" : 2,
    "n" : 3,
    "l" : [15, 10],
    "s" : [10, 2, 4],
    "distances" : [
        [0, 3, 3, 1],
        [3, 0, 4, 1],
        [3, 4, 0, 1],
        [3, 2, 1, 0]
    ]
}

instance12 = {
    "m" : 5,
    "n" : 10,
    "l" : [10, 10, 19, 12, 11],
    "s" : [6, 3, 2, 6, 4, 5, 3, 6, 4, 3],
    "distances" : [
                [ 0, 5, 3, 2, 2, 2, 4, 2, 3, 1, 5 ], 
                [ 2, 0, 1, 1, 1, 2, 2, 2, 3, 4, 2 ], 
                [ 4, 2, 0, 4, 2, 2, 2, 5, 4, 1, 2 ], 
                [ 4, 2, 3, 0, 3, 3, 3, 1, 5, 4, 1 ], 
                [ 4, 2, 4, 4, 0, 1, 4, 5, 2, 5, 2 ], 
                [ 2, 4, 5, 1, 3, 0, 1, 3, 5, 4, 3 ], 
                [ 5, 4, 3, 3, 5, 4, 0, 3, 5, 1, 2 ], 
                [ 4, 3, 2, 2, 5, 5, 2, 0, 5, 5, 1 ], 
                [ 1, 5, 3, 4, 1, 1, 4, 1, 0, 2, 3 ], 
                [ 4, 2, 2, 2, 1, 3, 4, 3, 1, 0, 4 ], 
                [ 4, 5, 5, 4, 3, 5, 5, 1, 5, 5, 0 ] 
            ]
}

In [140]:
def MCP(instance, upperBound, verbose=False):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of couriers
    distances = instance["distances"] # distances between packages

    solver = Solver()

    # Variables
    # To codify that courier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(n+2)] for j in range (n+1)] for i in range(m)]

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range (m):
        weigth_set = []
        for j in range (n):
            for k in range(1,n+1):
                for x in range(s[j]):
                    weigth_set.append(v[i][j][k])
        solver.add(at_most_k_seq(weigth_set, l[i], f"courier_{i}_load"))

    # 2. Each courier i starts and end at position j = n
    for i in range (m):
        solver.add(And(v[i][n][0], v[i][n][n+1]))


    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(n+2):
            solver.add(exactly_one_np([v[i][j][k] for j in range(n+1)], "amo_package"))
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_np([v[i][j][k] for k in range(1,n+1) for i in range(m)], "exactly_once"))
    
    # Symmetry breaking constraints
    # 5. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, n):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # symmetry breaking for courriers with equal capacity
    # ... TODO?

    # distance calculation
    for i in range (m):
        dist_set = []
        dist_set1 = []
        for j1 in range (n+1):
            for j2 in range (n+1):
                for k in range(1,n+2):
                    for x in range(distances[j1][j2]):
                        dist_set.append(And(v[i][j1][k-1],v[i][j2][k]))
                        dist_set1.append(And(v[i][j1][k-1],v[i][j2][k]))
        solver.add(at_most_k_seq(dist_set, upperBound, f"courier_{i}_distmax"))
    
    if not verbose:
        if solver.check() == sat:
            return solver.model()
        else:  
            return False
    else:
        print(solver.check())
        model = solver.model()
        if solver.check() == sat:
            for i in range(m):
                print()
                print(f"Courier {i+1}:")
                for k in range(n+2):
                    for j in range(n+1):
                        if model[v[i][j][k]]:
                            print(f"Time {k} Place {j} : {model[v[i][j][k]]}")


In [141]:
# Second version using Pseudo-Boolean constraints
def MCP_Pb(instance, upperBound, verbose=False):
    m = instance["m"] # courriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of courriers
    distances = instance["distances"] # distances between packages

    solver = Solver()

    # Variables
    # To codify that courrier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(n+2)] for j in range (n+1)] for i in range(m)]

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    # Pb version
    for i in range(m):
       solver.append(PbLe([(v[i][j][k],s[j]) for j in range(n) for k in range(1,n+1)], l[i]))


    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(And(v[i][n][0], v[i][n][n+1]))


    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(n+2):
            solver.add(exactly_one_np([v[i][j][k] for j in range(n+1)], "amo_package")) #(PbEq([(v[i][j][k],1) for j in range(n+1)],1))
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_np([v[i][j][k] for k in range(1,n+1) for i in range(m)], "exactly_once")) #(PbEq([(v[i][j][k],1) for k in range(1,n+1) for i in range(m)],1))    
    
    # Symmetry breaking constraints
    # 5. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, n):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # symmetry breaking for courriers with equal capacity
    # ... TODO?

    # distance calculation
    # Pb version
    for i in range(m):
        solver.append(PbLe([(And(v[i][j1][k-1],v[i][j2][k]),distances[j1][j2]) for j1 in range (n+1) for j2 in range (n+1) for k in range(1,n+2)], upperBound))

    if not verbose:
        if solver.check() == sat:
            return solver.model()
        else:  
            return False
    else:
        print(solver.check())
        model = solver.model()
        if solver.check() == sat:
            for i in range(m):
                print()
                print(f"Courier {i+1}:")
                for k in range(n+2):
                    for j in range(n+1):
                        if model[v[i][j][k]]:
                            print(f"Time {k} Place {j} : {model[v[i][j][k]]}")

In [142]:
def MultipleCouriersPlanning(instance, usePb = False):

    if usePb:
        mcp = MCP_Pb
    else:
        mcp = MCP

    n = instance["n"]
    distances = np.array(instance["distances"])

    upperBound1 = distances[n, 0] + np.sum(distances[np.arange(n), np.arange(n) + 1])
    upperBound2 = distances[0, n] + np.sum(distances[np.arange(n) + 1, np.arange(n)])
    upperBound = min(upperBound1, upperBound2)

    originalUpperBound = upperBound

    fromDepot = distances[n, :n]
    toDepot = distances[:n, n]
    inMiddle = distances[:n, :n]
    inMiddle = inMiddle[~np.eye(inMiddle.shape[0], dtype=bool)]
    numMiddle = int(np.ceil(instance["m"] / n)) - 1

    lowerBound = np.min(fromDepot) + np.min(toDepot) + numMiddle * np.min(inMiddle)
    pivot = (upperBound + lowerBound) // 2

    # binary search using bounds
    print("searching...")
    while True:
        if lowerBound == upperBound:
            return upperBound
        # print("lower:",lowerBound)
        # print("upper:",upperBound)
        print("pivot:",pivot)
        res = mcp(instance, pivot)
        if(res == False):
            print("fail")
            if lowerBound > upperBound:
                if upperBound == originalUpperBound:
                    return unsat
                else:
                    return upperBound
            lowerBound = pivot + 1
            pivot = (upperBound + lowerBound) // 2
        else:
            print("success")
            upperBound = pivot
            pivot = (upperBound + lowerBound) // 2

In [143]:
usePb = True
verbose = False
instance = instance12
result = MultipleCouriersPlanning(instance, usePb)
if result == unsat:
    print("\nResult of the computation: unsat")
else:    
    print("\nResult of the minimization: ",result)
    if verbose:
        if usePb:
            MCP_Pb(instance,result, verbose)
        else:
            MCP(instance,result, verbose)
    

searching...
pivot: 17
success
pivot: 9
success
pivot: 5
fail
pivot: 7
fail
pivot: 8
success

Result of the minimization:  8


In [144]:
MCP_Pb(instance12, 8, True)

sat

Courier 1:
Time 0 Place 10 : True
Time 1 Place 4 : True
Time 2 Place 8 : True
Time 3 Place 10 : True
Time 4 Place 10 : True
Time 5 Place 10 : True
Time 6 Place 10 : True
Time 7 Place 10 : True
Time 8 Place 10 : True
Time 9 Place 10 : True
Time 10 Place 10 : True
Time 11 Place 10 : True

Courier 2:
Time 0 Place 10 : True
Time 1 Place 5 : True
Time 2 Place 6 : True
Time 3 Place 10 : True
Time 4 Place 10 : True
Time 5 Place 10 : True
Time 6 Place 10 : True
Time 7 Place 10 : True
Time 8 Place 10 : True
Time 9 Place 10 : True
Time 10 Place 10 : True
Time 11 Place 10 : True

Courier 3:
Time 0 Place 10 : True
Time 1 Place 0 : True
Time 2 Place 9 : True
Time 3 Place 3 : True
Time 4 Place 10 : True
Time 5 Place 10 : True
Time 6 Place 10 : True
Time 7 Place 10 : True
Time 8 Place 10 : True
Time 9 Place 10 : True
Time 10 Place 10 : True
Time 11 Place 10 : True

Courier 4:
Time 0 Place 10 : True
Time 1 Place 10 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True
Time 5